<a href="https://colab.research.google.com/github/serinss/seSAC_ML_DL_Class/blob/main/05_2_%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%run my_init.py

Mounted at /content/drive


In [2]:
%run my_init.py

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#교차 검증
딥러닝 에서는 교차검증 하지 않음  
비정형 데이터 대상이므로 크기가 월등히 커서 시간이 많이 걸림  

교차 검증 장점 : 

1. 모든 데이터 셋을 평가에 활용할 수 있다.

  - 평가에 사용되는 데이터 편중을 막을 수 있다. 

(특정 평가 데이터 셋에 overfit 되는 것을 방지할 수 있다.)

  - 평가 결과에 따라 좀 더 일반화된 모델을 만들 수 있다. 

2. 모든 데이터 셋을 훈련에 활용할 수 있다.

  - 정확도를 향상시킬 수 있다.

  - 데이터 부족으로 인한 underfitting을 방지할 수 있다.

​

교차 검증 단점 : Iteration 횟수가 많기 때문에 모델 훈련/평가 시간이 오래 걸린다.

In [3]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol','sugar','pH']].to_numpy()
data.shape

(6497, 3)

In [5]:

target = wine['class'].to_numpy()
target.shape

(6497,)

In [6]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [7]:
train_input.shape, train_target.shape

((5197, 3), (5197,))

In [8]:
test_input.shape, test_target.shape

((1300, 3), (1300,))

In [9]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

In [10]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target, cv=3) # estimator, X, y=None
print(scores)

{'fit_time': array([0.01392579, 0.00655723, 0.0059793 ]), 'score_time': array([0.00098372, 0.00079703, 0.00076008]), 'test_score': array([0.84535488, 0.87182448, 0.84757506])}


In [11]:
np.mean(scores['test_score']) # 정확히는 validation score

0.8549181380146386

In [12]:
for k in scores.keys():
  print(k)

fit_time
score_time
test_score


In [13]:
scores = cross_validate(dt, train_input, train_target, cv=3,  return_train_score=True)
print(scores)

{'fit_time': array([0.01292777, 0.00617242, 0.00597882]), 'score_time': array([0.00132012, 0.00077581, 0.00078058]), 'test_score': array([0.84535488, 0.87182448, 0.84757506]), 'train_score': array([0.99711316, 0.9979798 , 0.9979798 ])}


In [14]:
np.mean(scores['test_score'])

0.8549181380146386

In [15]:
np.mean(scores['train_score']) #overfit 상태

0.9976909199772942

#K Fold 객체 사용

In [16]:
from sklearn.model_selection import KFold, StratifiedKFold

#n_splits : 몇 폴드로 교차검증을 할 지 선택 -> 무조건 첫번째 파라미터
kfold = KFold(3, shuffle=True, random_state=42) # 랜덤방식 -> 데이터가 많으면 편향이 발생하기 쉬움 -> 층화 임의 추출(StratifiedKFold)
skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) 

In [17]:
scores = cross_validate(dt, train_input, train_target, cv=kfold, return_train_score=True)
print(scores)

{'fit_time': array([0.00896072, 0.00860691, 0.00803518]), 'score_time': array([0.00132203, 0.00139713, 0.00085783]), 'test_score': array([0.84997115, 0.84237875, 0.84930716]), 'train_score': array([0.99711316, 0.9965368 , 0.9994228 ])}


In [18]:
scores = cross_validate(dt, train_input, train_target, cv=skfold, return_train_score=True)
print(scores)

{'fit_time': array([0.01053452, 0.00590348, 0.00623918]), 'score_time': array([0.00091934, 0.00082517, 0.00081587]), 'test_score': array([0.85920369, 0.85450346, 0.84237875]), 'train_score': array([0.99682448, 0.998557  , 0.9979798 ])}


In [19]:
for i,v in enumerate(np.arange(0.0003, 0.0008, 0.0001)):
  dt = DecisionTreeClassifier(min_impurity_decrease=v, random_state=42)
  scores = cross_validate(dt, train_input, train_target, cv=skfold, return_train_score=True )
  print(f'[{i}]')
  print(f"train score: {np.mean(scores['train_score'])}")
  print(f"test score: {np.mean(scores['test_score'])}", end = '\n\n')

# 점수들이 올라가다가 다시 내려감
  # min_impurity_decrease 값 0.0001 ~ 0.0010 이러한 값들은 경험적인 값들 -> 경험이 없으면 알 수 없음

[0]
train score: 0.9295742257058653
test score: 0.8637674370670857

[1]
train score: 0.9119681362175589
test score: 0.863958782267153

[2]
train score: 0.9000382412045229
test score: 0.8654987613091344

[3]
train score: 0.8947468748854431
test score: 0.8668459514554007

[4]
train score: 0.8869535562376208
test score: 0.8631911803966564



#하이퍼 파라미터 튜닝

In [20]:
from sklearn.model_selection import GridSearchCV

dt = DecisionTreeClassifier(random_state=42)
params = {
    "max_depth": range(3, 6),
    "min_samples_split": [2,5,8],
    "min_impurity_decrease": np.arange(0.0001, 0.0006, 0.0001)
}

In [21]:
3 * 3 * 5 * 5
# depth * split * impurity * fold 모두 곱하면 갯수가 나옴

225

In [22]:
gs = GridSearchCV(dt, params, cv=5, n_jobs=-1)
# estimator, param_grid * 별표 앞의 파라미터는 positional argument -> 순서대로 적어야함
# * 뒤에 오는 파라미터는 keyword argument -> 이름만 제대로 적으면 알아서 들어감

gs.fit(train_input, train_target)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(3, 6),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005]),
                         'min_samples_split': [2, 5, 8]})

In [23]:
gs.best_estimator_ #best 값은 저장을 해 두었으니 우리는 꺼내서 참조하면 됨

DecisionTreeClassifier(max_depth=5,
                       min_impurity_decrease=0.00030000000000000003,
                       random_state=42)

In [24]:
gs.best_params_

{'max_depth': 5,
 'min_impurity_decrease': 0.00030000000000000003,
 'min_samples_split': 2}

In [25]:
gs.best_score_ # test score

0.8585735174354039

In [26]:
gs.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [27]:
gs.cv_results_['mean_test_score']

array([0.84125583, 0.84125583, 0.84125583, 0.84125583, 0.84125583,
       0.84125583, 0.84125583, 0.84125583, 0.84125583, 0.84125583,
       0.84125583, 0.84125583, 0.84125583, 0.84125583, 0.84125583,
       0.85337806, 0.85337806, 0.85337806, 0.85337806, 0.85337806,
       0.85337806, 0.85337806, 0.85337806, 0.85337806, 0.85337806,
       0.85337806, 0.85337806, 0.85318557, 0.85318557, 0.85318557,
       0.85780355, 0.85780355, 0.85799604, 0.85799604, 0.85799604,
       0.85799604, 0.85857352, 0.85857352, 0.85857352, 0.85857352,
       0.85857352, 0.85857352, 0.85838102, 0.85838102, 0.85838102])

In [28]:
np.max(gs.cv_results_['mean_test_score']) # gs.best_score_ 와 동일

0.8585735174354039

In [29]:
best_model = gs.best_estimator_
best_model.get_params() #해당 베스트 모델의 hyper parameter 정보 꺼내오기

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.00030000000000000003,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

In [30]:
best_model.score(train_input, train_target) 

0.8668462574562248

In [31]:
best_model.score(test_input, test_target) # 아주 조금 낮음 - Decision Tree는 한계가 있음을 알 수 있음 -> 점수 잘 안오름
# overfit는 방지 했지만 점수 높지 않음

0.8623076923076923

In [32]:
# 책 예제

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # 5x5 = 25개 모델 fit()
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [33]:
print(gs.best_params_), print(gs.cv_results_['mean_test_score'])

{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


(None, None)

#확률 분포 선택 / 랜덤 서치
그리드 서치가 아니라 확률 분포를 이용한 서치를 실행

In [41]:
from scipy.stats import uniform, randint, norm, poisson, gamma, beta, bernoulli

In [43]:
ud = uniform(0,1)
ud.rvs(10, random_state=42) #랜덤하게 10개를 뽑겠다 - 복원추출

array([0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864,
       0.15599452, 0.05808361, 0.86617615, 0.60111501, 0.70807258])

In [44]:
ud.rvs((4, 5), random_state=42) #4행 5열

array([[0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864],
       [0.15599452, 0.05808361, 0.86617615, 0.60111501, 0.70807258],
       [0.02058449, 0.96990985, 0.83244264, 0.21233911, 0.18182497],
       [0.18340451, 0.30424224, 0.52475643, 0.43194502, 0.29122914]])

In [46]:
nd = norm(60, 5) # ~N(60,5)
# The location (loc=mean=평균값의 위치) keyword specifies the mean.
# The scale (scale=표준편차) keyword specifies the standard deviation.

nd.rvs((4, 6), random_state=42)

array([[62.48357077, 59.30867849, 63.23844269, 67.61514928, 58.82923313,
        58.82931522],
       [67.89606408, 63.83717365, 57.65262807, 62.71280022, 57.68291154,
        57.67135123],
       [61.20981136, 50.43359878, 51.37541084, 57.18856235, 54.9358444 ,
        61.57123666],
       [55.45987962, 52.93848149, 67.32824384, 58.8711185 , 60.33764102,
        52.87625907]])

In [52]:
params = {
    "max_depth": randint(3, 50),
    "min_samples_split": randint(2, 20),
    "min_impurity_decrease": uniform(0.00001, 0.001)
}

In [64]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(dt, params, n_iter=500, n_jobs=-1, cv=10, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(cv=10,
                   estimator=DecisionTreeClassifier(max_depth=42,
                                                    min_impurity_decrease=0.00023902539092479635,
                                                    min_samples_split=8,
                                                    random_state=42),
                   n_iter=500, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa0550fc5d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa0550ed5d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa0550fc2d0>},
                   random_state=42)

In [56]:
rs.best_score_ # CV가 적용 되었을 때의 점수 -> validation score (더 이상 의미 없음)

0.8731951237587076

In [57]:
rs.best_params_

{'max_depth': 42,
 'min_impurity_decrease': 0.00023902539092479635,
 'min_samples_split': 8}

In [62]:
dt = rs.best_estimator_
accuracy_score(train_target, dt.predict(train_input)) # train score

0.9141812584183182

In [61]:
accuracy_score(test_target, dt.predict(test_input)) # CV값 87 -> 86 best fit 임을 알 수 있음

0.8669230769230769